## **SETD2 with SAM**

## **Import modules**

In [1]:
import os
import sys

data_path=os.path.abspath('/media/dataHog/hca/SETD2') #Where your group data is
base_path=os.path.abspath('/media/dataHog/hca/SimFound_v2/source/') #Where your source code is (SFv2)

sys.path.append(base_path)
sys.path.append(data_path)

import importlib

import warnings
warnings.filterwarnings('ignore')
import Protocols as P

import main
import tools
import Featurize as F


from simtk.unit import *

## **Define Project**

In [2]:
importlib.reload(main)

workdir=data_path
results=workdir+'/results'
inputs=workdir+'inputs/structures/'

workdir=tools.Functions.fileHandler([workdir], _new=False)
tools.Functions.fileHandler([results, inputs])


#protein=['WT', 'R167Q', 'I602G']
protein=['setd2_open']

ligand=['SAM']
parameters=['310K']
timestep=20*picoseconds

project=main.Project(title='SETD2-SAM', 
                     hierarchy=('protein', 'ligand', 'parameter'), 
                     workdir=workdir,
                     parameter=parameters, 
                     replicas=15, 
                     protein=protein, 
                     ligand=ligand, 
                     results=results,
                     topology='wt_SETD2.pdb',
                     timestep=timestep,
                    initial_replica=11)


project_systems=project.setSystems()

Converted parameter "temperature" (in K) into scalar: 310.0 K
System defined: ('setd2_open', 'SAM', '310K', '11'), ID: 310.0 K
System defined: ('setd2_open', 'SAM', '310K', '12'), ID: 310.0 K
System defined: ('setd2_open', 'SAM', '310K', '13'), ID: 310.0 K
System defined: ('setd2_open', 'SAM', '310K', '14'), ID: 310.0 K
System defined: ('setd2_open', 'SAM', '310K', '15'), ID: 310.0 K


## **Simulation Protocol**

## Set defaults

In [3]:

SETD2_variants={('A',1499): 'CYX',
                ('A',1501): 'CYX',
                ('A',1516): 'CYX',
                ('A',1520): 'CYX',
                ('A',1529): 'CYX',
                ('A',1533):'CYX',
                ('A',1539):'CYX',
                ('A',1631):'CYX',
                ('A',1678):'CYX',
                ('A',1680):'CYX',
                ('A',1685):'CYX'}

NPT_equilibration={'ensemble': 'NPT',
                'step': 5*nanoseconds, 
                'report': timestep, 
                'restrained_sets': {'selections': ['protein and backbone', 'resname ZNB', 'resname SAM'],
                                    'forces': [100*kilojoules_per_mole/angstroms, 150*kilojoules_per_mole/angstroms, 150*kilojoules_per_mole/angstroms]}}

NPT_production={'ensemble': 'NPT',
                'step': 500*nanoseconds, 
                'report': timestep}

## Setup simulation protocol

In [4]:
importlib.reload(P)
importlib.reload(tools)
extra_pdb = ['ZNB_open.pdb', 'SAM.pdb']
#smiles = 'C[S+](CC[C@@H](C(=O)[O-])N)C[C@@H]1[C@H]([C@H]([C@@H](O1)N2C=NC3=C(N=CN=C32)N)O)O'
xml_files = ['protein.ff14SB.xml', 'tip4pew.xml', 'ZNB.xml', 'SAM.xml', 'bonded.xml', 'gaff.xml']




simulated_systems={}
for name, system in project_systems.items():

    if system.ligand == 'SAM':
    
        sim_tools=P.Protocols(workdir=system.name_folder, 
                              project_dir=project.workdir,
                              def_input_struct=project.def_input_struct,
                              def_input_ff=project.def_input_ff,
                             overwrite_mode=False)
    
        system_omm=sim_tools.pdb2omm(input_pdb=system.input_topology,
                                     extra_input_pdb=extra_pdb,
                                     ff_files=xml_files,
                                     protonate=True,
                                     solvate=True,
                                     pH_protein=7.0,
                                     name=name,
                                     residue_variants=SETD2_variants)
                                    #insert_molecules=True)
                                    #insert_smiles=smiles)
                                      
        simulated_systems[name]=system_omm


Setting openMM simulation protocols in /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate11
	Warning! Found conserved openMM system(s). /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate11/omm_1.pkl
	Overwrite False: /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate11/omm_2.pkl (1)


	Loaded openMM system: 
	File: /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate11/system.pdb, 
	Topology: <Topology; 5 chains, 22147 residues, 91663 atoms, 47929 bonds>
Setting openMM simulation protocols in /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate12
	Warning! Found conserved openMM system(s). /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate12/omm_1.pkl
	Overwrite False: /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate12/omm_2.pkl (1)
	Loaded openMM system: 
	File: /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate12/system.pdb, 
	Topology: <Topology; 5 chains, 22147 residues, 91663 atoms, 47929 bonds>
Setting openMM simulation protocols in /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate13
	Warning! Found conserved openMM system(s). /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate13/omm_1.pkl
	Overwrite False: /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate13/omm_2.pkl (1)
	Loaded openMM system: 
	File: /media/dataHog/hca/SETD2/se

## **WARNING!** Run simulation(s)

In [ ]:
importlib.reload(P)
importlib.reload(tools)

gpu_index='1'

simulations={}
for name, simulation in simulated_systems.items():  
    simulation.setSimulations(dt=0.002*picoseconds, 
                                  temperature = system.scalar,
                                  pH=7.0,
                                  friction = 1/picosecond,
                                  equilibrations=[NPT_equilibration],
                                  productions=[NPT_production],
                                  pressure=1*atmospheres,
                                  gpu_index=gpu_index)
    
    
    simulation.runSimulations(run_productions=True,  
                              compute_time=4,
                              reportFactor=0.025)        
    
    simulations[name]=simulation


The fastest platform is CUDA
Selected GPU ID: 1
Using platform CUDA, ID: 1
Run mode: 
	Overwrite: False
	Resume: True
	Compute time: 240 min


Energy minimization
	A complete checkpoint was found.
	System final potential energy: -1279635.4659479223 kJ/mol

EQ run 1: NPT
	A complete checkpoint was found.
	System final potential energy: -1034267.8098547924 kJ/mol

Production run 1: NPT
	A timeout checkpoint was found.
	PDB file generated: /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate11/production_NPT-1_init.pdb
	System initial potential energy: -1034267.7765725618 kJ/mol
	Simulation time: 404102.282 ps out of 500000.0 ps = 202051141/250000000 steps
	Simulating for 4:00:00 (0:06:00 checkpoints)...
	Warning! Simulation terminated by timeout: 6033750/250000000 = 243966250 steps to complete.
	PDB file generated: /media/dataHog/hca/SETD2/setd2_open-SAM-310K/replicate11/production_NPT-1_chk.pdb
	System final potential energy: -1035491.6053991271 kJ/mol
The fastest platform is CUDA
Sel

In [ ]:
simulations